In [1]:
import os
import matplotlib.pyplot as plt
import cv2

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D

# Using static random seed
np.random.seed(123)

# number of images taken from each
COUNT = 1000

Using TensorFlow backend.


#### Load Male Images

In [2]:
# Placeholder for data
male_list = list()

# Set the number taken from the male section
MALE_COUNT = COUNT

for filename in os.listdir("male/"):
    # Only use jpg files -- other types of files ignored
    if filename.endswith(".jpg"):
        
        relative_img_filename = os.path.join("male/", filename)
        
        # Create image from filename
        image = cv2.imread(relative_img_filename)
        resized_img = cv2.resize(image, dsize=(50, 50), interpolation=cv2.INTER_CUBIC)
        
        # Convert image to numpy array
        npImage = np.array(resized_img)
        
        # Add to list
        male_list.append(npImage)
        
        MALE_COUNT -= 1
        if(MALE_COUNT <= 0):
            break

# Turn male_list into numpy array
male_list = np.array(male_list)
male_list.shape

(1000, 50, 50, 3)

#### Load Female Images

In [3]:
female_list = list()

# Set the number taken from the female section
FEMALE_COUNT = COUNT

for filename in os.listdir("female/"):
    if filename.endswith(".jpg"):
        relative_img_filename = os.path.join("female/", filename)
        
        # Create image from filename
        image = cv2.imread(relative_img_filename)
        resized_img = cv2.resize(image, dsize=(50, 50), interpolation=cv2.INTER_CUBIC)
        
        # Convert image to numpy array
        npImage = np.array(resized_img)
        
        # Add to list prepending label (0: male, 1: female)
        female_list.append(npImage)
        
        FEMALE_COUNT -= 1
        if(FEMALE_COUNT <= 0):
            break

# Turn male_list into numpy array
female_list = np.array(female_list)
female_list.shape

(1000, 50, 50, 3)

#### Add labels (0: male, 1: female)

In [4]:
reshaped_male = list()
for item in male_list:
    reshaped_male.append((0, item))

reshaped_female = list()
for item in male_list:
    reshaped_female.append((1, item))

### Aggregate and Shuffle

In [5]:
all_data = reshaped_male + reshaped_female

In [6]:
from random import shuffle
shuffle(all_data)

### Arrange into Training / Test sets

In [7]:
# extract the labels
data = []
labels = []

In [8]:
for item in all_data:
    # Add label to labels
    labels.append(item[0])
    
    # Add data_item to data
    data.append(item[1])

In [9]:
data = np.array(data)

In [10]:
data.shape
# This should look like:
# (5500, 350, 350, 3)
# (num_imgs, width?, height?, depth) --> Depth signifies color depth or R, G, B as distinct values

(2000, 50, 50, 3)

In [11]:
labels = np.array(labels)

In [12]:
len(labels)

2000

#### Normalize pixel values
Current form:
- int value (0, 255)
Target form:
- float value (0.0, 1.0)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
# Split the data
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.20, random_state=42)

In [15]:
# data_train = data_train.astype('float32')

In [16]:
# Normalize values
data_train = data_train.astype('float32')
data_test = data_test.astype('float32')

data_train /= 255
data_test /= 255

## Build Model

In [17]:
model = Sequential()

#### Add layers

In [18]:
model.add(Conv2D(32, (3,3), input_shape=(50,50,3), activation='relu'))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(2, activation='sigmoid')) --> original
model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(1, activation='relu')) --> doesn't converge

#### Compile Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

#### Train Model

In [ ]:
model.fit(data_train,
          labels_train, 
          epochs=100,
          verbose=1)

Epoch 1/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.6931 - acc: 0.5012
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.6933 - acc: 0.5019
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.6931 - acc: 0.5100
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.6929 - acc: 0.5287
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.6930 - acc: 0.5038
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/step - loss: 0.6932 - acc: 0.5100
Epoch 7/100
1504/1600 [===========================>..] - ETA: 0s - loss: 0.6931 - acc: 0.5106